In [1]:
%pip install -U langchain langchain-openai langchain-huggingface langchain-community openai \
    faiss-cpu  unstructured pymupdf python-docx \
    sentence_transformers numpy pandas networkx openpyxl


  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     -------------------------------------- 981.5/981.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metada

  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
# conda install -c conda-forge faiss-gpu cudatoolkit=11.6

  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached scipy-1.16.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupS

ERROR: Could not install packages due to an OSError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'c:\\Users\\iwcyou\\.conda\\envs\\dev\\Lib\\site-packages\\mpmath\\tests\\test_functions.py'
Consider using the `--user` option or check the permissions.



In [1]:
from langchain.vectorstores import FAISS
import torch
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-zh",  # 或其他
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# from langchain_openai import OpenAIEmbeddings #需要openai的 API Key
# 如果需要使用 HuggingFace 的模型
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    PyPDFLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredExcelLoader,
)
import os

# 支持加载多种格式的文档
def load_documents_from_folder(folder_path: str):
    docs = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif filename.endswith(".docx"):
            loader = UnstructuredWordDocumentLoader(file_path)
        elif filename.endswith(".xlsx") or filename.endswith(".xls"):
            loader = UnstructuredExcelLoader(file_path)
        else:
            continue
        docs.extend(loader.load())
    return docs

# 切块 + 嵌入 + 存入 FAISS
def build_faiss_index_from_folder(folder_path: str, index_save_path: str):
    print(" 加载法规文档中...")
    raw_docs = load_documents_from_folder(folder_path)
    
    print(" 切分文档为片段...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    split_docs = splitter.split_documents(raw_docs)
    
    print(" 构建嵌入向量...")
    # embeddings = OpenAIEmbeddings()  # 或 HuggingFaceEmbeddings(model_name=\"...\")
    embeddings = embedding_model  

    print(" 构建 FAISS 向量数据库...")
    vectordb = FAISS.from_documents(split_docs, embeddings)

    print(f" 保存向量数据库至：{index_save_path}")
    vectordb.save_local(index_save_path)
    return vectordb


c:\Users\iwcyo\miniconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\iwcyo\miniconda3\envs\dev\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\iwcyo\.cache\huggingface\hub\models--BAAI--bge-small-zh. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer 

In [2]:
# 示例调用
folder = "./rule"
save_path = "./faiss_law_index"
build_faiss_index_from_folder(folder, save_path)

 加载法规文档中...
 切分文档为片段...
 构建嵌入向量...
 构建 FAISS 向量数据库...
 保存向量数据库至：./faiss_law_index


In [2]:
from langchain.vectorstores import FAISS
import torch
from langchain_huggingface import HuggingFaceEmbeddings

def query_law_index(query_text: str, index_path: str = "./faiss_law_index", k=3):
    vectordb = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True
)
    docs = vectordb.similarity_search(query_text, k=k)
    return "\n\n".join([d.page_content for d in docs])


In [3]:
state = {
    'filled_input': '运输企业在运输过程中，存在泥头车路面遗洒，责任如何划分？'
}
rag_context = query_law_index(state['filled_input'])

prompt = f"""
请参考以下法规内容作为决策依据:
{rag_context}

描述如下:
{state['filled_input']}

请判断责任归属及依据。
输出：
{{
  "responsibility": "运输企业",
  "reason": "......"
}}
"""


In [ ]:
import os
import openai
import sys
# sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['API_KEY']

In [7]:
import os
os.environ["OPENAI_API_KEY"] = "sk-3015211d76cd4c7dace6a833c90f2424"


In [8]:
from langchain.chat_models import ChatOpenAI  # 可替换为 deepseek-r1
from langchain.schema import HumanMessage
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

#  加载 FAISS 向量库
vectordb = FAISS.load_local(
    folder_path="./faiss_law_index",  # 你保存的知识库路径
    embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh"),
    allow_dangerous_deserialization=True
)

#  构造检索函数
def query_law_index(query_text: str, k=3):
    docs = vectordb.similarity_search(query_text, k=k)
    return "\n\n".join([doc.page_content for doc in docs])

#  构造 LangChain 模型（可替换为 DeepSeek）
# llm = ChatOpenAI(temperature=0, model="gpt-4")  # 或调用 deepseek-r1 的兼容封装

llm = ChatOpenAI(
    temperature=0,
    model="deepseek-chat",  # 可以使用 deepseek-chat、deepseek-coder 等
    openai_api_base="https://api.deepseek.com/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

#  模拟状态输入
state = {
    'filled_input': '运输企业在运输过程中，存在泥头车路面遗洒，责任如何划分？'
}

#  查询法规知识库
rag_context = query_law_index(state['filled_input'])

#  构造最终 Prompt
prompt = f"""
你是一位城市治理法规专家，请结合以下法规内容判断城市泥头车污染事件的责任归属。

法规内容如下：
{rag_context}

描述如下：
{state['filled_input']}

请判断责任归属，并说明理由。
输出 JSON 格式：
{{
  "responsibility": "运输企业",
  "reason": "......"
}}
"""

#  调用大模型进行回答
response = llm.invoke([HumanMessage(content=prompt)])

#  输出结果
print("📘 模型返回：")
print(response.content)


📘 模型返回：
```json
{
  "responsibility": "运输企业",
  "reason": "根据法规内容，运输企业作为整改主体，应确保泥头车在运输过程中遵守车辆操作规范和运输流程合规性。具体包括驾驶新型智能全密闭式泥头车，装载量不得超过核定标准，杜绝超载；每日出车前、收车后对车辆进行基本维护检查，并保持车内外整洁；按规定时间和路线行驶，进出工地前配合监管员完成车身冲洗，确保无带泥上路现象。运输企业在运输过程中存在泥头车路面遗洒，表明其未履行上述责任，因此责任归属于运输企业。"
}
```


In [ ]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

#  设置向量库路径
index_path = "./faiss_law_index"

# 加载向量库（本地，确保路径正确）
vectordb = FAISS.load_local(
    folder_path=index_path,
    embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh"),
    allow_dangerous_deserialization=True
)

#  1. 打印数据库信息
print("📦 向量库加载成功！")
print(f"📄 当前文档片段数：{len(vectordb.docstore._dict)}")

#  2. 显示前几条内容
print("\n🔍 文档片段预览（前 3 条）：")
for i, (doc_id, doc) in enumerate(vectordb.docstore._dict.items()):
    print(f"\n--- 文档 {i+1} ---")
    print(doc.page_content)
    if i >= 2:  # 只展示前3条
        break

#  3. 可选：执行一次关键字检索
query = "泥头车运输过程中未密闭造成路面污染，如何认定责任？"
results = vectordb.similarity_search(query, k=3)

print("\n🔎 示例检索结果：")
for i, r in enumerate(results):
    print(f"\n>>> Top-{i+1} 相关法规内容：")
    print(r.page_content)


📦 向量库加载成功！
📄 当前文档片段数：17

🔍 文档片段预览（前 3 条）：

--- 文档 1 ---
事件分类分级清单
（试行第一版）

--- 文档 2 ---
序号 一级分类 二级分类 三级分类\n事件描述 四级分类\n事件描述 要素 行业标准（法律（文件）依据） 区域（红线内、红线外、全域） 满足事项上报的时段 事件\n等级 圈层提级 环节 时限（小时/天） 主体 主体名称 岗位 人 职责 业务标准 考核标准 考核周期 备注\n（非必填） 1 城市管理 道路保洁 道路遗撒 泥头车遗撒 消防通道 《深圳经济特区市容和环境卫生管理条例》、《深圳市建筑废弃物管理办法》 全域 全天 A 自治 发现 / 自查主体 运输企业 监管员 1.落实车辆与驾驶员的资质管理\n2.动态监管（GPS）实时追踪泥头车\n3.安装安全装置（如全封闭式货箱）\n4.定期组织驾驶员安全培训

--- 文档 3 ---
全域 全天 A 自治 发现 / 自查主体 运输企业 监管员 1.落实车辆与驾驶员的资质管理\n2.动态监管（GPS）实时追踪泥头车\n3.安装安全装置（如全封闭式货箱）\n4.定期组织驾驶员安全培训 1.证件审核：驾驶员需持有与准驾车型匹配的驾驶证、特种作业证，并完成泥头车专项安全培训。车辆须具备行驶证、道路运输经营许可证、城市建筑垃圾准运证。\n2.动态监管：所有车辆强制安装带卫星定位功能的GPS设备，并接入统一监控平台。设置限速（高速公路70km/h、城市道路40km/h）、限定行驶路线及时间，实时预警超速、偏离路线等行为。\n3.封闭要求：货箱须采用全封闭平推式顶盖，尾门密封防漏撒，严禁超高超载。\n4.培训频率：新驾驶员岗前培训≥8小时，在职驾驶员每季度至少1次安全培训。 （一）正面清单：\n1.合法运营\n2.车辆安全规范\n3.车辆安全运输\n4.建立信息化管理\n（二）负面清单：\n1.无证运营\n2.车辆安全隐患\n3.车辆违规运输\n4.缺失信息化管理 / 自查主体 工地企业 监管员 1.选择合规运输企业\n2.施工现场管理\n3.周边环境保护

🔎 示例检索结果：

>>> Top-1 相关法规内容：
/ 监管主体 公安交警 / 强化路面运输管理，充分利用“雪亮工程”、道路查违摄像头、车牌号自动识别、不停车超限检测等技术手段，及时发现、查出建筑废弃